In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import struct
import random
import csv
from torch.utils.data import Dataset, DataLoader

selected_indices = set()
# -------------------------
# Definizione dei Modelli
# -------------------------

# Importazione dei modelli KAN
# Assicurati che il pacchetto 'kan_convolutional' sia installato e accessibile
try:
    from kan_convolutional.KANLinear import KANLinear
    import kan_convolutional.convolution
    from kan_convolutional.KANConv import KAN_Convolutional_Layer
except ImportError:
    raise ImportError("Il pacchetto 'kan_convolutional' non è stato trovato. Assicurati di averlo installato correttamente.")

class LeNet5(nn.Module):
    def __init__(self, num_classes=62):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)  # 28x28 -> 28x28
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)       # 28x28 -> 14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)             # 14x14 -> 10x10
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)       # 10x10 -> 5x5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)                   # Flatten
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16 * 5 * 5)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class LeNet5_KAN(nn.Module):
    def __init__(self, num_classes=62):  # EMNIST Balanced ha 62 classi
        super(LeNet5_KAN, self).__init__()
        
        # Primo strato conv: input=1 canale, output=6 filtri, kernel=5x5
        self.conv1 = KAN_Convolutional_Layer(
            in_channels=1,
            out_channels=6,
            kernel_size=(5,5),
            stride=(1,1),
            padding=(0,0),
            dilation=(1,1),
            grid_size=5,
            spline_order=3,
            scale_noise=0.1,
            scale_base=1.0,
            scale_spline=1.0,
            base_activation=torch.nn.ReLU,
            grid_eps=0.02,
            grid_range=(-1, 1)
        )
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Secondo strato conv: input=6 canali, output=16 filtri, kernel=5x5
        self.conv2 = KAN_Convolutional_Layer(
            in_channels=6,
            out_channels=16,
            kernel_size=(5,5),
            stride=(1,1),
            padding=(0,0),
            dilation=(1,1),
            grid_size=5,
            spline_order=3,
            scale_noise=0.1,
            scale_base=1.0,
            scale_spline=1.0,
            base_activation=torch.nn.ReLU,
            grid_eps=0.02,
            grid_range=(-1, 1)
        )
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        # Dopo conv1+pool1 (28x28 -> conv5x5->24x24 -> pool->12x12)
        # Dopo conv2+pool2 (12x12 -> conv5x5->8x8 -> pool->4x4)
        # 16 canali da 4x4 => 16*4*4=256
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)  

    def forward(self, x):
        # Passo 1: conv + pooling
        x = self.conv1(x)
        x = self.pool1(x)
        
        # Passo 2: conv + pooling
        x = self.conv2(x)
        x = self.pool2(x)
        
        # Flatten
        x = x.contiguous().view(x.size(0), -1)

        # Fully Connected Layers con ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Output Layer (senza attivazione)
        x = self.fc3(x)
        return x

# -------------------------
# Definizione del Dataset in Memoria
# -------------------------

class EMNISTMemoryDataset(Dataset):
    def __init__(self, data_tensor, labels_tensor):
        self.data = data_tensor
        self.labels = labels_tensor

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# -------------------------
# Funzioni per leggere i file IDX
# -------------------------

def read_idx_images(file_path):
    """Legge immagini in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
    return images

def read_idx_labels(file_path):
    """Legge etichette in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# -------------------------
# Funzione di Denormalizzazione
# -------------------------

def denormalize(tensor, mean, std):
    tensor = tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

# -------------------------
# Mappatura delle Classi
# -------------------------

def get_emnist_class_mapping():
    """
    Mappatura delle classi EMNIST ai caratteri corrispondenti.
    EMNIST ByClass ha 62 classi: 0-9, 10-35 A-Z, 36-61 a-z
    """
    characters = list("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
    return {i: char for i, char in enumerate(characters)}

# -------------------------
# Funzione per Estrarre un Campione Specifico
# -------------------------

def get_sample_by_index(dataset, index):
    """
    Estrae un campione specifico dal dataset utilizzando un indice.
    
    Args:
        dataset (Dataset): Il dataset da cui estrarre il campione.
        index (int): L'indice del campione da estrarre.
    
    Returns:
        tuple: (sample_data, sample_target)
    """
    if index < 0 or index >= len(dataset):
        raise IndexError("Indice fuori dal range del dataset.")
    sample_data, sample_target = dataset[index]
    return sample_data.unsqueeze(0), sample_target  # Aggiungi dimensione batch

# -------------------------
# Funzione per Generare e Salvataggio delle Classi Predette e Corrette
# -------------------------

def save_predicted_and_correct_class(sample_index, model, model_name, gradcam_dir, class_mapping, device, test_dataset, csv_writer):
    """
    Genera la predizione per un campione usando un modello e salva la classe predetta e corretta in un file CSV.
    
    Args:
        sample_index (int): Indice del campione nel dataset di test.
        model (nn.Module): Modello caricato.
        model_name (str): Nome del modello (es. "LeNet5: No Norm").
        gradcam_dir (str): Directory dove salvare il file CSV.
        class_mapping (dict): Dizionario mappatura indice classe -> carattere/numero.
        device (torch.device): Dispositivo (CPU o CUDA).
        test_dataset (Dataset): Dataset di test.
        csv_writer (csv.writer): Writer per il file CSV.
    """
    # Estrai il campione
    sample_data, sample_target = get_sample_by_index(test_dataset, sample_index)
    sample_data = sample_data.to(device)
    
    # Predizione
    with torch.no_grad():
        output = model(sample_data)
    pred_class_idx = output.argmax(dim=1).item()
    pred_class_char = class_mapping.get(pred_class_idx, "Unknown")
    
    # Classe corretta
    correct_class_idx = sample_target.item()
    correct_class_char = class_mapping.get(correct_class_idx, "Unknown")
    
    # Debugging: Stampa le classi per verifica
    print(f"Sample {sample_index} - Correct Class: {correct_class_char} (Index: {correct_class_idx}), Predicted Class: {pred_class_char} (Index: {pred_class_idx}) by {model_name}")
    
    # Scrivi nel CSV
    csv_writer.writerow([sample_index, correct_class_char, pred_class_char, model_name])

# -------------------------
# Funzione Principale
# -------------------------

def main():
    # Impostazioni
    learning_rate = 0.01
    optimizer_type = "SGD"
    grid_size = 5
    spline_order = 3
    norm_type = "L2"
    num_of_classes = 62
    batch_size = 1  # Rimuovere il batching

    seed = 12
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Device: {device}')

    mean, std = 0.1307, 0.3081

    data_dir = '/home/magliolo/.cache/emnist/gzip/'  # Modifica questo percorso se necessario

    # Definizione dei modelli e delle loro directory
    model_configs = [
        {
            "name": "LeNet5: No Norm",
            "gradcam_dir": "results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/",
            "model_dir": "results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/model/",
            "model_class": LeNet5
        },
        {
            "name": "KaNet5: No Norm",
            "gradcam_dir": "results/results_None_SGD_lr0.01_5_3/KaNet5/GradCAM/",
            "model_dir": "results/results_None_SGD_lr0.01_5_3/KaNet5/model/",
            "model_class": LeNet5_KAN
        },
        {
            "name": "LeNet5: L2 Norm",
            "gradcam_dir": "results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/",
            "model_dir": "results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/model/",
            "model_class": LeNet5
        },
        {
            "name": "KaNet5: L2 Norm",
            "gradcam_dir": "results/results_L2_SGD_lr0.01_5_3/KaNet5/GradCAM/",
            "model_dir": "results/results_L2_SGD_lr0.01_5_3/KaNet5/model/",
            "model_class": LeNet5_KAN
        }
    ]

    # Controllo che tutte le directory GradCAM esistano
    for config in model_configs:
        os.makedirs(config["gradcam_dir"], exist_ok=True)

    # Carica la mappatura delle classi
    class_mapping = get_emnist_class_mapping()

    # Carica i dati di test
    test_images_path = os.path.join(data_dir, 'emnist-byclass-test-images-idx3-ubyte')
    test_labels_path = os.path.join(data_dir, 'emnist-byclass-test-labels-idx1-ubyte')

    print("Leggendo i dati di test...")
    images_test = read_idx_images(test_images_path)
    labels_test = read_idx_labels(test_labels_path)

    # Converti in Tensori e normalizza
    test_images_tensor = torch.from_numpy(images_test.copy()).unsqueeze(1).float()
    test_labels_tensor = torch.from_numpy(labels_test.copy()).long()

    # Normalizzazione
    test_images_tensor = (test_images_tensor - mean) / std

    # Sposta su GPU
    test_images_tensor = test_images_tensor.to(device)
    test_labels_tensor = test_labels_tensor.to(device)

    # Crea il dataset in memoria
    test_dataset = EMNISTMemoryDataset(test_images_tensor, test_labels_tensor)

    # Crea il DataLoader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"Test size: {len(test_loader.dataset)}")
    print(f"Numero di classi uniche nel dataset di test: {len(set(labels_test))}")
    print(f"Etichette uniche nel dataset di test: {sorted(set(labels_test))}")

    # Definizione delle categorie richieste
    required_categories = {
        "misclass_model_1_only": [],
        "misclass_model_2_only": [],
        "misclass_model_3_only": [],
        "misclass_model_4_only": [],
        "misclass_all": [],
        "correct_all": []
    }

    # Numero di esempi per ogni categoria
    required_counts = {
        "misclass_model_1_only": 1,
        "misclass_model_2_only": 1,
        "misclass_model_3_only": 1,
        "misclass_model_4_only": 1,
        "misclass_all": 1,
        "correct_all": 2
    }

    # Flags per sapere quando abbiamo trovato tutto
    found = {
        "misclass_model_1_only": 0,
        "misclass_model_2_only": 0,
        "misclass_model_3_only": 0,
        "misclass_model_4_only": 0,
        "misclass_all": 0,
        "correct_all": 0
    }

    print("Inizio la ricerca dei campioni richiesti...")

    # Carica tutti i modelli
    loaded_models = []
    for config in model_configs:
        model_name = config["name"]
        gradcam_dir = config["gradcam_dir"]
        model_dir = config["model_dir"]
        model_class = config["model_class"]

        # Inizializza il modello
        model = model_class(num_classes=num_of_classes).to(device)

        # Trova il checkpoint_epoch_50.pth
        checkpoint_path = os.path.join(model_dir, "checkpoint_epoch_50.pth")
        if not os.path.exists(checkpoint_path):
            print(f"Checkpoint non trovato per {model_name} in {checkpoint_path}. Salto questo modello.")
            continue

        # Carica il checkpoint
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        epoch = checkpoint.get('epoch', 'Unknown')
        print(f"Checkpoint caricato per {model_name}: {checkpoint_path}, Epoch: {epoch}")
        model.eval()

        # Aggiungi il modello alla lista
        loaded_models.append({
            "name": model_name,
            "model": model,
            "gradcam_dir": gradcam_dir
        })

    if len(loaded_models) < 4:
        print("Non tutti i modelli sono stati caricati correttamente. Verifica i checkpoint e le configurazioni.")
        return

    # Itera su ogni campione del dataset di test
    for idx in range(len(test_dataset)):
        if all(found[cat] >= required_counts[cat] for cat in required_categories):
            print("Tutti i campioni richiesti sono stati trovati.")
            break

        sample_data, sample_target = get_sample_by_index(test_dataset, idx)
        sample_data = sample_data.to(device)

        # Predizioni di tutti i modelli
        pred_classes = []
        for loaded_model in loaded_models:
            model = loaded_model["model"]
            with torch.no_grad():
                output = model(sample_data)
            pred_class_idx = output.argmax(dim=1).item()
            pred_class_char = class_mapping.get(pred_class_idx, "Unknown")
            pred_classes.append(pred_class_char)

        # Classe corretta
        correct_class_idx = sample_target.item()
        correct_class_char = class_mapping.get(correct_class_idx, "Unknown")

        # Determina quali modelli hanno sbagliato
        misclass_flags = [pred != correct_class_char for pred in pred_classes]

        # Categorie specifiche
        misclass_model_1 = misclass_flags[0]
        misclass_model_2 = misclass_flags[1]
        misclass_model_3 = misclass_flags[2]
        misclass_model_4 = misclass_flags[3]

        # Controlla le categorie
        if (misclass_model_1 and not misclass_model_2 and not misclass_model_3 and not misclass_model_4 and
            found["misclass_model_1_only"] < required_counts["misclass_model_1_only"]):
            required_categories["misclass_model_1_only"].append(idx)
            found["misclass_model_1_only"] += 1
            print(f"Trovato campione {idx} che sbaglia solo il modello 1.")
            continue

        if (misclass_model_2 and not misclass_model_1 and not misclass_model_3 and not misclass_model_4 and
            found["misclass_model_2_only"] < required_counts["misclass_model_2_only"]):
            required_categories["misclass_model_2_only"].append(idx)
            found["misclass_model_2_only"] += 1
            print(f"Trovato campione {idx} che sbaglia solo il modello 2.")
            continue

        if (misclass_model_3 and not misclass_model_1 and not misclass_model_2 and not misclass_model_4 and
            found["misclass_model_3_only"] < required_counts["misclass_model_3_only"]):
            required_categories["misclass_model_3_only"].append(idx)
            found["misclass_model_3_only"] += 1
            print(f"Trovato campione {idx} che sbaglia solo il modello 3.")
            continue

        if (misclass_model_4 and not misclass_model_1 and not misclass_model_2 and not misclass_model_3 and
            found["misclass_model_4_only"] < required_counts["misclass_model_4_only"]):
            required_categories["misclass_model_4_only"].append(idx)
            found["misclass_model_4_only"] += 1
            print(f"Trovato campione {idx} che sbaglia solo il modello 4.")
            continue

        # Campione che sbaglia tutti i modelli
        if (misclass_model_1 and misclass_model_2 and misclass_model_3 and misclass_model_4 and
            found["misclass_all"] < required_counts["misclass_all"]):
            required_categories["misclass_all"].append(idx)
            found["misclass_all"] += 1
            print(f"Trovato campione {idx} che sbaglia tutti i modelli.")
            continue

        # Campione che non sbaglia nessun modello
        if (not misclass_model_1 and not misclass_model_2 and not misclass_model_3 and not misclass_model_4 and
            found["correct_all"] < required_counts["correct_all"]):
            required_categories["correct_all"].append(idx)
            found["correct_all"] += 1
            print(f"Trovato campione {idx} che non sbaglia nessun modello.")
            continue

    # Verifica se tutte le categorie sono state trovate
    for cat, count in required_counts.items():
        if found[cat] < count:
            print(f"Attenzione: Solo {found[cat]} campioni trovati per la categoria '{cat}', richiesta {count}.")
        else:
            print(f"Tutti i campioni richiesti per la categoria '{cat}' sono stati trovati.")

    # Unisci tutti gli indici raccolti
    global selected_indices
    for cat in required_categories:
        selected_indices.update(required_categories[cat])

    selected_indices = sorted(list(selected_indices))
    print(f"\nIndici dei campioni selezionati: {selected_indices}\n")

    # Itera su ogni modello e salva le predizioni e classi corrette per i campioni selezionati
    for loaded_model in loaded_models:
        model_name = loaded_model["name"]
        model = loaded_model["model"]
        gradcam_dir = loaded_model["gradcam_dir"]

        # Prepara il file CSV
        csv_filename = "predictions.csv"
        csv_path = os.path.join(gradcam_dir, csv_filename)
        with open(csv_path, mode='w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            # Scrivi l'intestazione
            csv_writer.writerow(["sample_index", "correct_class", "predicted_class", "model_name"])

            # Itera su ogni sample_index
            for sample_index in selected_indices:
                try:
                    save_predicted_and_correct_class(
                        sample_index=sample_index,
                        model=model,
                        model_name=model_name,
                        gradcam_dir=gradcam_dir,
                        class_mapping=class_mapping,
                        device=device,
                        test_dataset=test_dataset,
                        csv_writer=csv_writer
                    )
                except Exception as e:
                    print(f"Errore nella predizione per sample {sample_index} con {model_name}: {e}")

        print(f"Predizioni salvate in: {csv_path}")

if __name__ == "__main__":
    main()


Device: cuda
Leggendo i dati di test...
Test size: 116323
Numero di classi uniche nel dataset di test: 62
Etichette uniche nel dataset di test: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]
Inizio la ricerca dei campioni richiesti...
Checkpoint caricato per LeNet5: No Norm: results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Checkpoint caricato per KaNet5: No Norm: results/results_None_SGD_lr0.01_5_3/KaNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Checkpoint caricato per LeNet5: L2 Norm: results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Checkpoint caricato per KaNet5: L2 Norm: results/results_L2_SGD_lr0.01_5_3/KaNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Trovato campione 0 che sbaglia tutti i modelli.
Trovato cam

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import struct
import random

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# -------------------------
# Definizione del Modello
# -------------------------

class LeNet5(nn.Module):
    def __init__(self, num_classes=62):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)  # 28x28 -> 28x28
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)       # 28x28 -> 14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)             # 14x14 -> 10x10
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)       # 10x10 -> 5x5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)                   # Flatten
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16 * 5 * 5)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# -------------------------
# Definizione del Dataset in Memoria
# -------------------------

class EMNISTMemoryDataset(Dataset):
    def __init__(self, data_tensor, labels_tensor):
        self.data = data_tensor
        self.labels = labels_tensor

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# -------------------------
# Funzioni per leggere i file IDX
# -------------------------

def read_idx_images(file_path):
    """Legge immagini in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
    return images

def read_idx_labels(file_path):
    """Legge etichette in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# -------------------------
# Funzione di Denormalizzazione
# -------------------------

def denormalize(tensor, mean, std):
    tensor = tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

# -------------------------
# Mappatura delle Classi
# -------------------------

def get_emnist_class_mapping():
    """
    Mappatura delle classi EMNIST ai caratteri corrispondenti.
    EMNIST ByClass ha 62 classi: 0-9, 10-35 A-Z, 36-61 a-z
    """
    characters = list("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
    return {i: char for i, char in enumerate(characters)}

# -------------------------
# Funzione per Estrarre un Campione Specifico
# -------------------------

def get_sample_by_index(dataset, index):
    """
    Estrae un campione specifico dal dataset utilizzando un indice.
    
    Args:
        dataset (Dataset): Il dataset da cui estrarre il campione.
        index (int): L'indice del campione da estrarre.
    
    Returns:
        tuple: (sample_data, sample_target)
    """
    if index < 0 or index >= len(dataset):
        raise IndexError("Indice fuori dal range del dataset.")
    sample_data, sample_target = dataset[index]
    return sample_data.unsqueeze(0), sample_target  # Aggiungi dimensione batch

# -------------------------
# Funzione per Sovrapporre la Heatmap con un Alpha Regolabile
# -------------------------

def overlay_heatmap_on_image(original, heatmap, alpha=0.6, colormap='jet'):
    """
    Sovrappone una heatmap su un'immagine originale con un parametro alpha regolabile.
    
    Args:
        original (numpy.ndarray): Immagine originale in formato RGB e normalizzata tra 0 e 1.
        heatmap (numpy.ndarray): Heatmap normalizzata tra 0 e 1.
        alpha (float): Trasparenza della heatmap.
        colormap (str): Colormap da utilizzare per la heatmap.
    
    Returns:
        numpy.ndarray: Immagine con la heatmap sovrapposta.
    """
    import cv2
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB) / 255.0  # Converti da BGR a RGB e normalizza
    overlay = (1 - alpha) * original + alpha * heatmap_color
    overlay = np.clip(overlay, 0, 1)
    return overlay

# -------------------------
# Funzione per Generare e Visualizzare Grad-CAM
# -------------------------

def generate_gradcam_plots(sample_index, gradcam_dir):
    """
    Genera e salva le heatmap Grad-CAM per un campione specifico.
    
    Args:
        sample_index (int): L'indice del campione da analizzare.
        gradcam_dir (str): Directory dove salvare le immagini Grad-CAM.
    """
    # Verifica che l'indice sia valido
    if sample_index < 0 or sample_index >= len(test_dataset):
        raise IndexError(f"Indice {sample_index} fuori dal range del dataset.")
    
    # Estrai il campione specifico
    sample_data, sample_target = get_sample_by_index(test_dataset, sample_index)
    sample_data = sample_data.to(device)
    sample_target = sample_target.item()
    
    # Definisci il target layer
    target_layer = model.conv2
    
    # Inizializza GradCAM senza use_cuda
    cam = GradCAM(model=model, target_layers=[target_layer])
    
    # Genera la heatmap Grad-CAM
    grayscale_cam = cam(input_tensor=sample_data, targets=None)  # None = classe predetta
    grayscale_cam = grayscale_cam[0]  # Rimuovi dimensione batch
    
    # Normalizza la heatmap tra 0 e 1
    grayscale_cam_normalized = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min() + 1e-8)
    
    # Ottieni la classe predetta
    with torch.no_grad():
        output = model(sample_data)
    pred_class = output.argmax(dim=1).item()
    
    # Denormalizza l'immagine originale per visualizzazione
    original_image = denormalize(sample_data.cpu().clone(), [mean], [std]).squeeze().numpy()
    
    # Flip orizzontale e rotazione di 90 gradi anti-clockwise
    original_image = np.fliplr(original_image)  # Flip orizzontale
    original_image = np.rot90(original_image, k=1)  # Rotazione di 90 gradi anti-clockwise
    
    grayscale_cam_normalized = np.fliplr(grayscale_cam_normalized)  # Flip orizzontale della heatmap
    grayscale_cam_normalized = np.rot90(grayscale_cam_normalized, k=1)  # Rotazione di 90 gradi anti-clockwise della heatmap
    
    # Converti l'immagine originale in RGB e normalizzala tra 0 e 1
    original_image_rgb = np.stack([original_image]*3, axis=2)  # [H, W, 3]
    original_image_rgb = original_image_rgb / 255.0  # Assicurati che l'immagine sia tra 0 e 1
    
    # Sovrapponi manualmente la heatmap con un alpha maggiore
    visualization = overlay_heatmap_on_image(original_image_rgb, grayscale_cam_normalized, alpha=0.6)
    
    # Salva separatamente le tre tipologie di immagini
    
    # 1. Immagine Originale
    original_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_original.png')
    plt.imsave(original_save_path, original_image, cmap='gray')
    
    # 2. Heatmap Grad-CAM
    heatmap_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_heatmap.png')
    plt.imsave(heatmap_save_path, grayscale_cam_normalized, cmap='jet')
    
    # 3. Immagine con Heatmap Sovrapposta
    overlay_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_overlay.png')
    plt.imsave(overlay_save_path, visualization)
    
    # 4. Grafico Completo
    fig, axs = plt.subplots(1, 3, figsize=(24, 8))
    
    # Plot 1: Immagine Originale
    axs[0].imshow(original_image, cmap='gray')
    axs[0].set_title(f"Immagine Originale - Classe Vera: {class_mapping[sample_target]}")
    axs[0].axis('off')
    
    # Plot 2: Heatmap Grad-CAM
    im = axs[1].imshow(grayscale_cam_normalized, cmap='jet')
    axs[1].set_title("Heatmap Grad-CAM")
    axs[1].axis('off')
    # Aggiungi una barra laterale (colorbar)
    cbar = fig.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel('Intensità', rotation=270, labelpad=15)
    
    # Plot 3: Immagine con Heatmap Sovrapposta
    axs[2].imshow(visualization)
    axs[2].set_title(f"Classe Predetta: {class_mapping[pred_class]}")
    axs[2].axis('off')
    
    # Salva il grafico completo
    chart_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_chart.png')
    plt.tight_layout()
    plt.savefig(chart_save_path)
    plt.close(fig)  # Chiudi la figura per liberare memoria
    
    print(f"Immagine Originale salvata in: {original_save_path}")
    print(f"Heatmap Grad-CAM salvata in: {heatmap_save_path}")
    print(f"Immagine con Heatmap Sovrapposta salvata in: {overlay_save_path}")
    print(f"Grafico Completo salvato in: {chart_save_path}")
    
    # Pulisci i hook dopo aver finito
    cam = None  # Libera risorse (GradCAM chiama automaticamente remove hooks nel suo metodo __del__)

# -------------------------
# Funzione Principale
# -------------------------

def main():
    global model, test_dataset, class_mapping, mean, std, base_dir, device

    # Impostazione dei parametri fissi
    learning_rate = 0.01
    optimizer_type = "SGD"
    grid_size = 0
    spline_order = 0
    norm_type = "None"
    num_of_classes = 62
    batch_size = 1  # Rimuovere il batching

    # Seme per riproducibilità
    seed = 12
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Device: {device}')

    mean, std = 0.1307, 0.3081

    data_dir = '/home/magliolo/.cache/emnist/gzip/'

    base_dir = os.path.join(
        'results',
        f"results_{norm_type}_{optimizer_type}_lr{learning_rate}_{grid_size}_{spline_order}",
        'Standard_LeNet5'
    )
    model_dir = os.path.join(base_dir, "model")

    if not os.path.exists(model_dir):
        raise FileNotFoundError(f"Directory del modello non trovata: {model_dir}")

    model = LeNet5(num_classes=num_of_classes).to(device)

    checkpoints = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
    if not checkpoints:
        raise FileNotFoundError(f"Nessun checkpoint trovato nella directory: {model_dir}")
    try:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    except ValueError:
        # Se il formato del checkpoint è diverso, usa semplicemente il file più recente
        latest_checkpoint = max(checkpoints, key=lambda x: os.path.getctime(os.path.join(model_dir, x)))
    checkpoint_path = os.path.join(model_dir, latest_checkpoint)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Checkpoint caricato: {checkpoint_path}, Epoch: {checkpoint.get('epoch', 'Unknown')}")
    model.eval()

    # Leggi i dati di test
    test_images_path = os.path.join(data_dir, 'emnist-byclass-test-images-idx3-ubyte')
    test_labels_path = os.path.join(data_dir, 'emnist-byclass-test-labels-idx1-ubyte')

    print("Leggendo i dati di test...")
    images_test = read_idx_images(test_images_path)
    labels_test = read_idx_labels(test_labels_path)

    # Converti in Tensori e normalizza
    test_images_tensor = torch.from_numpy(images_test.copy()).unsqueeze(1).float()
    test_labels_tensor = torch.from_numpy(labels_test.copy()).long()

    # Normalizzazione
    test_images_tensor = (test_images_tensor - mean) / std

    # Sposta su GPU
    test_images_tensor = test_images_tensor.to(device)
    test_labels_tensor = test_labels_tensor.to(device)

    # Crea il dataset in memoria
    test_dataset = EMNISTMemoryDataset(test_images_tensor, test_labels_tensor)

    # Crea il DataLoader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"Test size: {len(test_loader.dataset)}")
    print(f"Numero di classi uniche nel dataset di test: {len(set(labels_test))}")
    print(f"Etichette uniche nel dataset di test: {sorted(set(labels_test))}")

    # Mappatura delle classi
    class_mapping = get_emnist_class_mapping()

    # Crea la subfolder "GradCAM" all'interno di base_dir
    gradcam_dir = os.path.join(base_dir, "GradCAM")
    os.makedirs(gradcam_dir, exist_ok=True)
    print(f"Directory per GradCAM: {gradcam_dir}")

    # Ora puoi chiamare la funzione `generate_gradcam_plots` con l'indice desiderato
    # Esempio:
    sample_indices = selected_indices  # Puoi aggiungere altri indici qui

    for sample_index in sample_indices:
        print(f"\nGenerando Grad-CAM per l'indice: {sample_index}")
        try:
            generate_gradcam_plots(sample_index, gradcam_dir)
        except Exception as e:
            print(f"Errore nella generazione della heatmap per l'indice {sample_index}: {e}")

if __name__ == "__main__":
    main()


Device: cuda
Checkpoint caricato: results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Leggendo i dati di test...
Test size: 116323
Numero di classi uniche nel dataset di test: 62
Etichette uniche nel dataset di test: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]
Directory per GradCAM: results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM

Generando Grad-CAM per l'indice: 0
Immagine Originale salvata in: results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/sample_0_original.png
Heatmap Grad-CAM salvata in: results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/sample_0_heatmap.png
Immagine con Heatmap Sovrapposta salvata in: results/results_None_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/sample_0_overlay.png
Grafico Completo salvato in: result

In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import struct
import random

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# -------------------------
# Definizione del Modello
# -------------------------

class LeNet5(nn.Module):
    def __init__(self, num_classes=62):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)  # 28x28 -> 28x28
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)       # 28x28 -> 14x14
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)             # 14x14 -> 10x10
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)       # 10x10 -> 5x5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)                   # Flatten
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16 * 5 * 5)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# -------------------------
# Definizione del Dataset in Memoria
# -------------------------

class EMNISTMemoryDataset(Dataset):
    def __init__(self, data_tensor, labels_tensor):
        self.data = data_tensor
        self.labels = labels_tensor

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# -------------------------
# Funzioni per leggere i file IDX
# -------------------------

def read_idx_images(file_path):
    """Legge immagini in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
    return images

def read_idx_labels(file_path):
    """Legge etichette in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# -------------------------
# Funzione di Denormalizzazione
# -------------------------

def denormalize(tensor, mean, std):
    tensor = tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

# -------------------------
# Mappatura delle Classi
# -------------------------

def get_emnist_class_mapping():
    """
    Mappatura delle classi EMNIST ai caratteri corrispondenti.
    EMNIST ByClass ha 62 classi: 0-9, 10-35 A-Z, 36-61 a-z
    """
    characters = list("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
    return {i: char for i, char in enumerate(characters)}

# -------------------------
# Funzione per Estrarre un Campione Specifico
# -------------------------

def get_sample_by_index(dataset, index):
    """
    Estrae un campione specifico dal dataset utilizzando un indice.
    
    Args:
        dataset (Dataset): Il dataset da cui estrarre il campione.
        index (int): L'indice del campione da estrarre.
    
    Returns:
        tuple: (sample_data, sample_target)
    """
    if index < 0 or index >= len(dataset):
        raise IndexError("Indice fuori dal range del dataset.")
    sample_data, sample_target = dataset[index]
    return sample_data.unsqueeze(0), sample_target  # Aggiungi dimensione batch

# -------------------------
# Funzione per Sovrapporre la Heatmap con un Alpha Regolabile
# -------------------------

def overlay_heatmap_on_image(original, heatmap, alpha=0.6, colormap='jet'):
    """
    Sovrappone una heatmap su un'immagine originale con un parametro alpha regolabile.
    
    Args:
        original (numpy.ndarray): Immagine originale in formato RGB e normalizzata tra 0 e 1.
        heatmap (numpy.ndarray): Heatmap normalizzata tra 0 e 1.
        alpha (float): Trasparenza della heatmap.
        colormap (str): Colormap da utilizzare per la heatmap.
    
    Returns:
        numpy.ndarray: Immagine con la heatmap sovrapposta.
    """
    import cv2
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB) / 255.0  # Converti da BGR a RGB e normalizza
    overlay = (1 - alpha) * original + alpha * heatmap_color
    overlay = np.clip(overlay, 0, 1)
    return overlay

# -------------------------
# Funzione per Generare e Visualizzare Grad-CAM
# -------------------------

def generate_gradcam_plots(sample_index, gradcam_dir):
    """
    Genera e salva le heatmap Grad-CAM per un campione specifico.
    
    Args:
        sample_index (int): L'indice del campione da analizzare.
        gradcam_dir (str): Directory dove salvare le immagini Grad-CAM.
    """
    # Verifica che l'indice sia valido
    if sample_index < 0 or sample_index >= len(test_dataset):
        raise IndexError(f"Indice {sample_index} fuori dal range del dataset.")
    
    # Estrai il campione specifico
    sample_data, sample_target = get_sample_by_index(test_dataset, sample_index)
    sample_data = sample_data.to(device)
    sample_target = sample_target.item()
    
    # Definisci il target layer
    target_layer = model.conv2
    
    # Inizializza GradCAM senza use_cuda
    cam = GradCAM(model=model, target_layers=[target_layer])
    
    # Genera la heatmap Grad-CAM
    grayscale_cam = cam(input_tensor=sample_data, targets=None)  # None = classe predetta
    grayscale_cam = grayscale_cam[0]  # Rimuovi dimensione batch
    
    # Normalizza la heatmap tra 0 e 1
    grayscale_cam_normalized = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min() + 1e-8)
    
    # Ottieni la classe predetta
    with torch.no_grad():
        output = model(sample_data)
    pred_class = output.argmax(dim=1).item()
    
    # Denormalizza l'immagine originale per visualizzazione
    original_image = denormalize(sample_data.cpu().clone(), [mean], [std]).squeeze().numpy()
    
    # Flip orizzontale e rotazione di 90 gradi anti-clockwise
    original_image = np.fliplr(original_image)  # Flip orizzontale
    original_image = np.rot90(original_image, k=1)  # Rotazione di 90 gradi anti-clockwise
    
    grayscale_cam_normalized = np.fliplr(grayscale_cam_normalized)  # Flip orizzontale della heatmap
    grayscale_cam_normalized = np.rot90(grayscale_cam_normalized, k=1)  # Rotazione di 90 gradi anti-clockwise della heatmap
    
    # Converti l'immagine originale in RGB e normalizzala tra 0 e 1
    original_image_rgb = np.stack([original_image]*3, axis=2)  # [H, W, 3]
    original_image_rgb = original_image_rgb / 255.0  # Assicurati che l'immagine sia tra 0 e 1
    
    # Sovrapponi manualmente la heatmap con un alpha maggiore
    visualization = overlay_heatmap_on_image(original_image_rgb, grayscale_cam_normalized, alpha=0.6)
    
    # Salva separatamente le tre tipologie di immagini
    
    # 1. Immagine Originale
    original_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_original.png')
    plt.imsave(original_save_path, original_image, cmap='gray')
    
    # 2. Heatmap Grad-CAM
    heatmap_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_heatmap.png')
    plt.imsave(heatmap_save_path, grayscale_cam_normalized, cmap='jet')
    
    # 3. Immagine con Heatmap Sovrapposta
    overlay_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_overlay.png')
    plt.imsave(overlay_save_path, visualization)
    
    # 4. Grafico Completo
    fig, axs = plt.subplots(1, 3, figsize=(24, 8))
    
    # Plot 1: Immagine Originale
    axs[0].imshow(original_image, cmap='gray')
    axs[0].set_title(f"Immagine Originale - Classe Vera: {class_mapping[sample_target]}")
    axs[0].axis('off')
    
    # Plot 2: Heatmap Grad-CAM
    im = axs[1].imshow(grayscale_cam_normalized, cmap='jet')
    axs[1].set_title("Heatmap Grad-CAM")
    axs[1].axis('off')
    # Aggiungi una barra laterale (colorbar)
    cbar = fig.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel('Intensità', rotation=270, labelpad=15)
    
    # Plot 3: Immagine con Heatmap Sovrapposta
    axs[2].imshow(visualization)
    axs[2].set_title(f"Classe Predetta: {class_mapping[pred_class]}")
    axs[2].axis('off')
    
    # Salva il grafico completo
    chart_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_chart.png')
    plt.tight_layout()
    plt.savefig(chart_save_path)
    plt.close(fig)  # Chiudi la figura per liberare memoria
    
    print(f"Immagine Originale salvata in: {original_save_path}")
    print(f"Heatmap Grad-CAM salvata in: {heatmap_save_path}")
    print(f"Immagine con Heatmap Sovrapposta salvata in: {overlay_save_path}")
    print(f"Grafico Completo salvato in: {chart_save_path}")
    
    # Pulisci i hook dopo aver finito
    cam = None  # Libera risorse (GradCAM chiama automaticamente remove hooks nel suo metodo __del__)

# -------------------------
# Funzione Principale
# -------------------------

def main():
    global model, test_dataset, class_mapping, mean, std, base_dir, device

    # Impostazione dei parametri fissi
    learning_rate = 0.01
    optimizer_type = "SGD"
    grid_size = 0
    spline_order = 0
    norm_type = "L2"
    num_of_classes = 62
    batch_size = 1  # Rimuovere il batching

    # Seme per riproducibilità
    seed = 12
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Device: {device}')

    mean, std = 0.1307, 0.3081

    data_dir = '/home/magliolo/.cache/emnist/gzip/'

    base_dir = os.path.join(
        'results',
        f"results_{norm_type}_{optimizer_type}_lr{learning_rate}_{grid_size}_{spline_order}",
        'Standard_LeNet5'
    )
    model_dir = os.path.join(base_dir, "model")

    if not os.path.exists(model_dir):
        raise FileNotFoundError(f"Directory del modello non trovata: {model_dir}")

    model = LeNet5(num_classes=num_of_classes).to(device)

    checkpoints = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
    if not checkpoints:
        raise FileNotFoundError(f"Nessun checkpoint trovato nella directory: {model_dir}")
    try:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    except ValueError:
        # Se il formato del checkpoint è diverso, usa semplicemente il file più recente
        latest_checkpoint = max(checkpoints, key=lambda x: os.path.getctime(os.path.join(model_dir, x)))
    checkpoint_path = os.path.join(model_dir, latest_checkpoint)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Checkpoint caricato: {checkpoint_path}, Epoch: {checkpoint.get('epoch', 'Unknown')}")
    model.eval()

    # Leggi i dati di test
    test_images_path = os.path.join(data_dir, 'emnist-byclass-test-images-idx3-ubyte')
    test_labels_path = os.path.join(data_dir, 'emnist-byclass-test-labels-idx1-ubyte')

    print("Leggendo i dati di test...")
    images_test = read_idx_images(test_images_path)
    labels_test = read_idx_labels(test_labels_path)

    # Converti in Tensori e normalizza
    test_images_tensor = torch.from_numpy(images_test.copy()).unsqueeze(1).float()
    test_labels_tensor = torch.from_numpy(labels_test.copy()).long()

    # Normalizzazione
    test_images_tensor = (test_images_tensor - mean) / std

    # Sposta su GPU
    test_images_tensor = test_images_tensor.to(device)
    test_labels_tensor = test_labels_tensor.to(device)

    # Crea il dataset in memoria
    test_dataset = EMNISTMemoryDataset(test_images_tensor, test_labels_tensor)

    # Crea il DataLoader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"Test size: {len(test_loader.dataset)}")
    print(f"Numero di classi uniche nel dataset di test: {len(set(labels_test))}")
    print(f"Etichette uniche nel dataset di test: {sorted(set(labels_test))}")

    # Mappatura delle classi
    class_mapping = get_emnist_class_mapping()

    # Crea la subfolder "GradCAM" all'interno di base_dir
    gradcam_dir = os.path.join(base_dir, "GradCAM")
    os.makedirs(gradcam_dir, exist_ok=True)
    print(f"Directory per GradCAM: {gradcam_dir}")

    # Ora puoi chiamare la funzione `generate_gradcam_plots` con l'indice desiderato
    # Esempio:
    sample_indices = selected_indices  # Puoi aggiungere altri indici qui

    for sample_index in sample_indices:
        print(f"\nGenerando Grad-CAM per l'indice: {sample_index}")
        try:
            generate_gradcam_plots(sample_index, gradcam_dir)
        except Exception as e:
            print(f"Errore nella generazione della heatmap per l'indice {sample_index}: {e}")

if __name__ == "__main__":
    main()


Device: cuda
Checkpoint caricato: results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Leggendo i dati di test...
Test size: 116323
Numero di classi uniche nel dataset di test: 62
Etichette uniche nel dataset di test: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]
Directory per GradCAM: results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM

Generando Grad-CAM per l'indice: 0
Immagine Originale salvata in: results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/sample_0_original.png
Heatmap Grad-CAM salvata in: results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/sample_0_heatmap.png
Immagine con Heatmap Sovrapposta salvata in: results/results_L2_SGD_lr0.01_0_0/Standard_LeNet5/GradCAM/sample_0_overlay.png
Grafico Completo salvato in: results/results_

In [4]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import struct
import random

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# -------------------------
# Definizione del Modello KAN
# -------------------------

from kan_convolutional.KANLinear import KANLinear
import kan_convolutional.convolution
from kan_convolutional.KANConv import KAN_Convolutional_Layer

class LeNet5_KAN(nn.Module):
    def __init__(self, num_classes=62):  # EMNIST Balanced ha 62 classi
        super(LeNet5_KAN, self).__init__()
        
        # Primo strato conv: input=1 canale, output=6 filtri, kernel=5x5
        self.conv1 = KAN_Convolutional_Layer(
            in_channels=1,
            out_channels=6,
            kernel_size=(5,5),
            stride=(1,1),
            padding=(0,0),
            dilation=(1,1),
            grid_size=5,
            spline_order=3,
            scale_noise=0.1,
            scale_base=1.0,
            scale_spline=1.0,
            base_activation=torch.nn.ReLU,
            grid_eps=0.02,
            grid_range=(-1, 1)
        )
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Secondo strato conv: input=6 canali, output=16 filtri, kernel=5x5
        self.conv2 = KAN_Convolutional_Layer(
            in_channels=6,
            out_channels=16,
            kernel_size=(5,5),
            stride=(1,1),
            padding=(0,0),
            dilation=(1,1),
            grid_size=5,
            spline_order=3,
            scale_noise=0.1,
            scale_base=1.0,
            scale_spline=1.0,
            base_activation=torch.nn.ReLU,
            grid_eps=0.02,
            grid_range=(-1, 1)
        )
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        # Dopo conv1+pool1 (28x28 -> conv5x5->24x24 -> pool->12x12)
        # Dopo conv2+pool2 (12x12 -> conv5x5->8x8 -> pool->4x4)
        # 16 canali da 4x4 => 16*4*4=256
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)  

    def forward(self, x):
        # Passo 1: conv + pooling
        x = self.conv1(x)
        x = self.pool1(x)
        
        # Passo 2: conv + pooling
        x = self.conv2(x)
        x = self.pool2(x)
        
        # Flatten
        x = x.contiguous().view(x.size(0), -1)

        # Fully Connected Layers con ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Output Layer (senza attivazione)
        x = self.fc3(x)
        return x

# -------------------------
# Definizione del Dataset in Memoria
# -------------------------

class EMNISTMemoryDataset(Dataset):
    def __init__(self, data_tensor, labels_tensor):
        self.data = data_tensor
        self.labels = labels_tensor

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# -------------------------
# Funzioni per leggere i file IDX
# -------------------------

def read_idx_images(file_path):
    """Legge immagini in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
    return images

def read_idx_labels(file_path):
    """Legge etichette in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# -------------------------
# Funzione di Denormalizzazione
# -------------------------

def denormalize(tensor, mean, std):
    tensor = tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

# -------------------------
# Mappatura delle Classi
# -------------------------

def get_emnist_class_mapping():
    characters = list("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
    return {i: char for i, char in enumerate(characters)}

# -------------------------
# Funzione per Estrarre un Campione Specifico
# -------------------------

def get_sample_by_index(dataset, index):
    if index < 0 or index >= len(dataset):
        raise IndexError("Indice fuori dal range del dataset.")
    sample_data, sample_target = dataset[index]
    return sample_data.unsqueeze(0), sample_target  # Aggiungi dimensione batch

# -------------------------
# Funzione per Sovrapporre la Heatmap con un Alpha Regolabile
# -------------------------

def overlay_heatmap_on_image(original, heatmap, alpha=0.6, colormap='jet'):
    """
    Sovrappone una heatmap su un'immagine originale con un parametro alpha regolabile.
    
    Args:
        original (numpy.ndarray): Immagine originale in formato RGB e normalizzata tra 0 e 1.
        heatmap (numpy.ndarray): Heatmap normalizzata tra 0 e 1.
        alpha (float): Trasparenza della heatmap.
        colormap (str): Colormap da utilizzare per la heatmap.
    
    Returns:
        numpy.ndarray: Immagine con la heatmap sovrapposta.
    """
    import cv2
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB) / 255.0  # Converti da BGR a RGB e normalizza
    overlay = (1 - alpha) * original + alpha * heatmap_color
    overlay = np.clip(overlay, 0, 1)
    return overlay

# -------------------------
# Funzione per Generare e Visualizzare Grad-CAM
# -------------------------

def generate_gradcam_plots(sample_index, gradcam_dir):
    """
    Genera e salva le heatmap Grad-CAM per un campione specifico.
    
    Args:
        sample_index (int): L'indice del campione da analizzare.
        gradcam_dir (str): Directory dove salvare le immagini Grad-CAM.
    """
    # Verifica che l'indice sia valido
    if sample_index < 0 or sample_index >= len(test_dataset):
        raise IndexError(f"Indice {sample_index} fuori dal range del dataset.")
    
    # Estrai il campione specifico
    sample_data, sample_target = get_sample_by_index(test_dataset, sample_index)
    sample_data = sample_data.to(device)
    sample_target = sample_target.item()
    
    # Definisci il target layer
    target_layer = model.conv2
    
    # Inizializza GradCAM senza use_cuda
    cam = GradCAM(model=model, target_layers=[target_layer])
    
    # Genera la heatmap Grad-CAM
    grayscale_cam = cam(input_tensor=sample_data, targets=None)  # None = classe predetta
    grayscale_cam = grayscale_cam[0]  # Rimuovi dimensione batch
    
    # Normalizza la heatmap tra 0 e 1
    grayscale_cam_normalized = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min() + 1e-8)
    
    # Ottieni la classe predetta
    with torch.no_grad():
        output = model(sample_data)
    pred_class = output.argmax(dim=1).item()
    
    # Denormalizza l'immagine originale per visualizzazione
    original_image = denormalize(sample_data.cpu().clone(), [mean], [std]).squeeze().numpy()
    
    # Flip orizzontale e rotazione di 90 gradi anti-clockwise
    original_image = np.fliplr(original_image)  # Flip orizzontale
    original_image = np.rot90(original_image, k=1)  # Rotazione di 90 gradi anti-clockwise
    
    grayscale_cam_normalized = np.fliplr(grayscale_cam_normalized)  # Flip orizzontale della heatmap
    grayscale_cam_normalized = np.rot90(grayscale_cam_normalized, k=1)  # Rotazione di 90 gradi anti-clockwise della heatmap
    
    # Converti l'immagine originale in RGB e normalizzala tra 0 e 1
    original_image_rgb = np.stack([original_image]*3, axis=2)  # [H, W, 3]
    original_image_rgb = original_image_rgb / 255.0  # Assicurati che l'immagine sia tra 0 e 1
    
    # Sovrapponi manualmente la heatmap con un alpha maggiore
    visualization = overlay_heatmap_on_image(original_image_rgb, grayscale_cam_normalized, alpha=0.6)
    
    # Salva separatamente le quattro tipologie di immagini
    
    # 1. Immagine Originale
    original_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_original.png')
    plt.imsave(original_save_path, original_image, cmap='gray')
    
    # 2. Heatmap Grad-CAM
    heatmap_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_heatmap.png')
    plt.imsave(heatmap_save_path, grayscale_cam_normalized, cmap='jet')
    
    # 3. Immagine con Heatmap Sovrapposta
    overlay_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_overlay.png')
    plt.imsave(overlay_save_path, visualization)
    
    # 4. Grafico Completo
    fig, axs = plt.subplots(1, 3, figsize=(24, 8))
    
    # Plot 1: Immagine Originale
    axs[0].imshow(original_image, cmap='gray')
    axs[0].set_title(f"Immagine Originale - Classe Vera: {class_mapping[sample_target]}")
    axs[0].axis('off')
    
    # Plot 2: Heatmap Grad-CAM
    im = axs[1].imshow(grayscale_cam_normalized, cmap='jet')
    axs[1].set_title("Heatmap Grad-CAM")
    axs[1].axis('off')
    # Aggiungi una barra laterale (colorbar)
    cbar = fig.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel('Intensità', rotation=270, labelpad=15)
    
    # Plot 3: Immagine con Heatmap Sovrapposta
    axs[2].imshow(visualization)
    axs[2].set_title(f"Classe Predetta: {class_mapping[pred_class]}")
    axs[2].axis('off')
    
    # Salva il grafico completo
    chart_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_chart.png')
    plt.tight_layout()
    plt.savefig(chart_save_path)
    plt.close(fig)  # Chiudi la figura per liberare memoria
    
    print(f"Immagine Originale salvata in: {original_save_path}")
    print(f"Heatmap Grad-CAM salvata in: {heatmap_save_path}")
    print(f"Immagine con Heatmap Sovrapposta salvata in: {overlay_save_path}")
    print(f"Grafico Completo salvato in: {chart_save_path}")
    
    # Pulisci i hook dopo aver finito
    cam = None  # Libera risorse (GradCAM chiama automaticamente remove hooks nel suo metodo __del__)

# -------------------------
# Funzione Principale
# -------------------------

def main():
    global model, test_dataset, class_mapping, mean, std, base_dir, device

    # Impostazioni
    learning_rate = 0.01
    optimizer_type = "SGD"
    grid_size = 5
    spline_order = 3
    norm_type = "L2"
    num_of_classes = 62
    batch_size = 1  # Rimuovere il batching

    seed = 12
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Device: {device}')

    mean, std = 0.1307, 0.3081

    data_dir = '/home/magliolo/.cache/emnist/gzip/'

    base_dir = os.path.join(
        'results',
        f"results_{norm_type}_{optimizer_type}_lr{learning_rate}_{grid_size}_{spline_order}",
        'KaNet5'
    )
    model_dir = os.path.join(base_dir, "model")

    if not os.path.exists(model_dir):
        raise FileNotFoundError(f"Directory del modello non trovata: {model_dir}")

    # Utilizzo del modello KAN invece del classico LeNet5
    model = LeNet5_KAN(num_classes=num_of_classes).to(device)

    checkpoints = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
    if not checkpoints:
        raise FileNotFoundError(f"Nessun checkpoint trovato nella directory: {model_dir}")
    try:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    except ValueError:
        # Se il formato del checkpoint è diverso, usa semplicemente il file più recente
        latest_checkpoint = max(checkpoints, key=lambda x: os.path.getctime(os.path.join(model_dir, x)))
    checkpoint_path = os.path.join(model_dir, latest_checkpoint)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Checkpoint caricato: {checkpoint_path}, Epoch: {checkpoint.get('epoch', 'Unknown')}")
    model.eval()

    # Leggi i dati di test
    test_images_path = os.path.join(data_dir, 'emnist-byclass-test-images-idx3-ubyte')
    test_labels_path = os.path.join(data_dir, 'emnist-byclass-test-labels-idx1-ubyte')

    print("Leggendo i dati di test...")
    images_test = read_idx_images(test_images_path)
    labels_test = read_idx_labels(test_labels_path)

    # Converti in Tensori e normalizza
    test_images_tensor = torch.from_numpy(images_test.copy()).unsqueeze(1).float()
    test_labels_tensor = torch.from_numpy(labels_test.copy()).long()

    # Normalizzazione
    test_images_tensor = (test_images_tensor - mean) / std

    # Sposta su GPU
    test_images_tensor = test_images_tensor.to(device)
    test_labels_tensor = test_labels_tensor.to(device)

    # Crea il dataset in memoria
    test_dataset = EMNISTMemoryDataset(test_images_tensor, test_labels_tensor)

    # Crea il DataLoader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"Test size: {len(test_loader.dataset)}")
    print(f"Numero di classi uniche nel dataset di test: {len(set(labels_test))}")
    print(f"Etichette uniche nel dataset di test: {sorted(set(labels_test))}")

    # Mappatura delle classi
    class_mapping = get_emnist_class_mapping()

    # Crea la subfolder "GradCAM" all'interno di base_dir
    gradcam_dir = os.path.join(base_dir, "GradCAM")
    os.makedirs(gradcam_dir, exist_ok=True)
    print(f"Directory per GradCAM: {gradcam_dir}")

    # Indici di esempio
    sample_indices = selected_indices # Puoi aggiungere altri indici qui

    for sample_index in sample_indices:
        print(f"\nGenerando Grad-CAM per l'indice: {sample_index}")
        try:
            generate_gradcam_plots(sample_index, gradcam_dir)
        except Exception as e:
            print(f"Errore nella generazione della heatmap per l'indice {sample_index}: {e}")

if __name__ == "__main__":
    main()


Device: cuda
Checkpoint caricato: results/results_L2_SGD_lr0.01_5_3/KaNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Leggendo i dati di test...
Test size: 116323
Numero di classi uniche nel dataset di test: 62
Etichette uniche nel dataset di test: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]
Directory per GradCAM: results/results_L2_SGD_lr0.01_5_3/KaNet5/GradCAM

Generando Grad-CAM per l'indice: 0
Immagine Originale salvata in: results/results_L2_SGD_lr0.01_5_3/KaNet5/GradCAM/sample_0_original.png
Heatmap Grad-CAM salvata in: results/results_L2_SGD_lr0.01_5_3/KaNet5/GradCAM/sample_0_heatmap.png
Immagine con Heatmap Sovrapposta salvata in: results/results_L2_SGD_lr0.01_5_3/KaNet5/GradCAM/sample_0_overlay.png
Grafico Completo salvato in: results/results_L2_SGD_lr0.01_5_3/KaNet5/GradCAM/sample_0_cha

In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import struct
import random

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# -------------------------
# Definizione del Modello KAN
# -------------------------

from kan_convolutional.KANLinear import KANLinear
import kan_convolutional.convolution
from kan_convolutional.KANConv import KAN_Convolutional_Layer

class LeNet5_KAN(nn.Module):
    def __init__(self, num_classes=62):  # EMNIST Balanced ha 62 classi
        super(LeNet5_KAN, self).__init__()
        
        # Primo strato conv: input=1 canale, output=6 filtri, kernel=5x5
        self.conv1 = KAN_Convolutional_Layer(
            in_channels=1,
            out_channels=6,
            kernel_size=(5,5),
            stride=(1,1),
            padding=(0,0),
            dilation=(1,1),
            grid_size=5,
            spline_order=3,
            scale_noise=0.1,
            scale_base=1.0,
            scale_spline=1.0,
            base_activation=torch.nn.ReLU,
            grid_eps=0.02,
            grid_range=(-1, 1)
        )
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # Secondo strato conv: input=6 canali, output=16 filtri, kernel=5x5
        self.conv2 = KAN_Convolutional_Layer(
            in_channels=6,
            out_channels=16,
            kernel_size=(5,5),
            stride=(1,1),
            padding=(0,0),
            dilation=(1,1),
            grid_size=5,
            spline_order=3,
            scale_noise=0.1,
            scale_base=1.0,
            scale_spline=1.0,
            base_activation=torch.nn.ReLU,
            grid_eps=0.02,
            grid_range=(-1, 1)
        )
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        # Dopo conv1+pool1 (28x28 -> conv5x5->24x24 -> pool->12x12)
        # Dopo conv2+pool2 (12x12 -> conv5x5->8x8 -> pool->4x4)
        # 16 canali da 4x4 => 16*4*4=256
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)  

    def forward(self, x):
        # Passo 1: conv + pooling
        x = self.conv1(x)
        x = self.pool1(x)
        
        # Passo 2: conv + pooling
        x = self.conv2(x)
        x = self.pool2(x)
        
        # Flatten
        x = x.contiguous().view(x.size(0), -1)

        # Fully Connected Layers con ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Output Layer (senza attivazione)
        x = self.fc3(x)
        return x

# -------------------------
# Definizione del Dataset in Memoria
# -------------------------

class EMNISTMemoryDataset(Dataset):
    def __init__(self, data_tensor, labels_tensor):
        self.data = data_tensor
        self.labels = labels_tensor

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# -------------------------
# Funzioni per leggere i file IDX
# -------------------------

def read_idx_images(file_path):
    """Legge immagini in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
    return images

def read_idx_labels(file_path):
    """Legge etichette in formato IDX."""
    with open(file_path, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# -------------------------
# Funzione di Denormalizzazione
# -------------------------

def denormalize(tensor, mean, std):
    tensor = tensor.clone()
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

# -------------------------
# Mappatura delle Classi
# -------------------------

def get_emnist_class_mapping():
    characters = list("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")
    return {i: char for i, char in enumerate(characters)}

# -------------------------
# Funzione per Estrarre un Campione Specifico
# -------------------------

def get_sample_by_index(dataset, index):
    if index < 0 or index >= len(dataset):
        raise IndexError("Indice fuori dal range del dataset.")
    sample_data, sample_target = dataset[index]
    return sample_data.unsqueeze(0), sample_target  # Aggiungi dimensione batch

# -------------------------
# Funzione per Sovrapporre la Heatmap con un Alpha Regolabile
# -------------------------

def overlay_heatmap_on_image(original, heatmap, alpha=0.6, colormap='jet'):
    """
    Sovrappone una heatmap su un'immagine originale con un parametro alpha regolabile.
    
    Args:
        original (numpy.ndarray): Immagine originale in formato RGB e normalizzata tra 0 e 1.
        heatmap (numpy.ndarray): Heatmap normalizzata tra 0 e 1.
        alpha (float): Trasparenza della heatmap.
        colormap (str): Colormap da utilizzare per la heatmap.
    
    Returns:
        numpy.ndarray: Immagine con la heatmap sovrapposta.
    """
    import cv2
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB) / 255.0  # Converti da BGR a RGB e normalizza
    overlay = (1 - alpha) * original + alpha * heatmap_color
    overlay = np.clip(overlay, 0, 1)
    return overlay

# -------------------------
# Funzione per Generare e Visualizzare Grad-CAM
# -------------------------

def generate_gradcam_plots(sample_index, gradcam_dir):
    """
    Genera e salva le heatmap Grad-CAM per un campione specifico.
    
    Args:
        sample_index (int): L'indice del campione da analizzare.
        gradcam_dir (str): Directory dove salvare le immagini Grad-CAM.
    """
    # Verifica che l'indice sia valido
    if sample_index < 0 or sample_index >= len(test_dataset):
        raise IndexError(f"Indice {sample_index} fuori dal range del dataset.")
    
    # Estrai il campione specifico
    sample_data, sample_target = get_sample_by_index(test_dataset, sample_index)
    sample_data = sample_data.to(device)
    sample_target = sample_target.item()
    
    # Definisci il target layer
    target_layer = model.conv2
    
    # Inizializza GradCAM senza use_cuda
    cam = GradCAM(model=model, target_layers=[target_layer])
    
    # Genera la heatmap Grad-CAM
    grayscale_cam = cam(input_tensor=sample_data, targets=None)  # None = classe predetta
    grayscale_cam = grayscale_cam[0]  # Rimuovi dimensione batch
    
    # Normalizza la heatmap tra 0 e 1
    grayscale_cam_normalized = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min() + 1e-8)
    
    # Ottieni la classe predetta
    with torch.no_grad():
        output = model(sample_data)
    pred_class = output.argmax(dim=1).item()
    
    # Denormalizza l'immagine originale per visualizzazione
    original_image = denormalize(sample_data.cpu().clone(), [mean], [std]).squeeze().numpy()
    
    # Flip orizzontale e rotazione di 90 gradi anti-clockwise
    original_image = np.fliplr(original_image)  # Flip orizzontale
    original_image = np.rot90(original_image, k=1)  # Rotazione di 90 gradi anti-clockwise
    
    grayscale_cam_normalized = np.fliplr(grayscale_cam_normalized)  # Flip orizzontale della heatmap
    grayscale_cam_normalized = np.rot90(grayscale_cam_normalized, k=1)  # Rotazione di 90 gradi anti-clockwise della heatmap
    
    # Converti l'immagine originale in RGB e normalizzala tra 0 e 1
    original_image_rgb = np.stack([original_image]*3, axis=2)  # [H, W, 3]
    original_image_rgb = original_image_rgb / 255.0  # Assicurati che l'immagine sia tra 0 e 1
    
    # Sovrapponi manualmente la heatmap con un alpha maggiore
    visualization = overlay_heatmap_on_image(original_image_rgb, grayscale_cam_normalized, alpha=0.6)
    
    # Salva separatamente le quattro tipologie di immagini
    
    # 1. Immagine Originale
    original_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_original.png')
    plt.imsave(original_save_path, original_image, cmap='gray')
    
    # 2. Heatmap Grad-CAM
    heatmap_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_heatmap.png')
    plt.imsave(heatmap_save_path, grayscale_cam_normalized, cmap='jet')
    
    # 3. Immagine con Heatmap Sovrapposta
    overlay_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_overlay.png')
    plt.imsave(overlay_save_path, visualization)
    
    # 4. Grafico Completo
    fig, axs = plt.subplots(1, 3, figsize=(24, 8))
    
    # Plot 1: Immagine Originale
    axs[0].imshow(original_image, cmap='gray')
    axs[0].set_title(f"Immagine Originale - Classe Vera: {class_mapping[sample_target]}")
    axs[0].axis('off')
    
    # Plot 2: Heatmap Grad-CAM
    im = axs[1].imshow(grayscale_cam_normalized, cmap='jet')
    axs[1].set_title("Heatmap Grad-CAM")
    axs[1].axis('off')
    # Aggiungi una barra laterale (colorbar)
    cbar = fig.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel('Intensità', rotation=270, labelpad=15)
    
    # Plot 3: Immagine con Heatmap Sovrapposta
    axs[2].imshow(visualization)
    axs[2].set_title(f"Classe Predetta: {class_mapping[pred_class]}")
    axs[2].axis('off')
    
    # Salva il grafico completo
    chart_save_path = os.path.join(gradcam_dir, f'sample_{sample_index}_chart.png')
    plt.tight_layout()
    plt.savefig(chart_save_path)
    plt.close(fig)  # Chiudi la figura per liberare memoria
    
    print(f"Immagine Originale salvata in: {original_save_path}")
    print(f"Heatmap Grad-CAM salvata in: {heatmap_save_path}")
    print(f"Immagine con Heatmap Sovrapposta salvata in: {overlay_save_path}")
    print(f"Grafico Completo salvato in: {chart_save_path}")
    
    # Pulisci i hook dopo aver finito
    cam = None  # Libera risorse (GradCAM chiama automaticamente remove hooks nel suo metodo __del__)

# -------------------------
# Funzione Principale
# -------------------------

def main():
    global model, test_dataset, class_mapping, mean, std, base_dir, device

    # Impostazioni
    learning_rate = 0.01
    optimizer_type = "SGD"
    grid_size = 5
    spline_order = 3
    norm_type = "None"
    num_of_classes = 62
    batch_size = 1  # Rimuovere il batching

    seed = 12
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Device: {device}')

    mean, std = 0.1307, 0.3081

    data_dir = '/home/magliolo/.cache/emnist/gzip/'

    base_dir = os.path.join(
        'results',
        f"results_{norm_type}_{optimizer_type}_lr{learning_rate}_{grid_size}_{spline_order}",
        'KaNet5'
    )
    model_dir = os.path.join(base_dir, "model")

    if not os.path.exists(model_dir):
        raise FileNotFoundError(f"Directory del modello non trovata: {model_dir}")

    # Utilizzo del modello KAN invece del classico LeNet5
    model = LeNet5_KAN(num_classes=num_of_classes).to(device)

    checkpoints = [f for f in os.listdir(model_dir) if f.endswith('.pth')]
    if not checkpoints:
        raise FileNotFoundError(f"Nessun checkpoint trovato nella directory: {model_dir}")
    try:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    except ValueError:
        # Se il formato del checkpoint è diverso, usa semplicemente il file più recente
        latest_checkpoint = max(checkpoints, key=lambda x: os.path.getctime(os.path.join(model_dir, x)))
    checkpoint_path = os.path.join(model_dir, latest_checkpoint)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Checkpoint caricato: {checkpoint_path}, Epoch: {checkpoint.get('epoch', 'Unknown')}")
    model.eval()

    # Leggi i dati di test
    test_images_path = os.path.join(data_dir, 'emnist-byclass-test-images-idx3-ubyte')
    test_labels_path = os.path.join(data_dir, 'emnist-byclass-test-labels-idx1-ubyte')

    print("Leggendo i dati di test...")
    images_test = read_idx_images(test_images_path)
    labels_test = read_idx_labels(test_labels_path)

    # Converti in Tensori e normalizza
    test_images_tensor = torch.from_numpy(images_test.copy()).unsqueeze(1).float()
    test_labels_tensor = torch.from_numpy(labels_test.copy()).long()

    # Normalizzazione
    test_images_tensor = (test_images_tensor - mean) / std

    # Sposta su GPU
    test_images_tensor = test_images_tensor.to(device)
    test_labels_tensor = test_labels_tensor.to(device)

    # Crea il dataset in memoria
    test_dataset = EMNISTMemoryDataset(test_images_tensor, test_labels_tensor)

    # Crea il DataLoader
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"Test size: {len(test_loader.dataset)}")
    print(f"Numero di classi uniche nel dataset di test: {len(set(labels_test))}")
    print(f"Etichette uniche nel dataset di test: {sorted(set(labels_test))}")

    # Mappatura delle classi
    class_mapping = get_emnist_class_mapping()

    # Crea la subfolder "GradCAM" all'interno di base_dir
    gradcam_dir = os.path.join(base_dir, "GradCAM")
    os.makedirs(gradcam_dir, exist_ok=True)
    print(f"Directory per GradCAM: {gradcam_dir}")

    # Indici di esempio
    sample_indices = selected_indices  # Puoi aggiungere altri indici qui

    for sample_index in sample_indices:
        print(f"\nGenerando Grad-CAM per l'indice: {sample_index}")
        try:
            generate_gradcam_plots(sample_index, gradcam_dir)
        except Exception as e:
            print(f"Errore nella generazione della heatmap per l'indice {sample_index}: {e}")

if __name__ == "__main__":
    main()


Device: cuda
Checkpoint caricato: results/results_None_SGD_lr0.01_5_3/KaNet5/model/checkpoint_epoch_50.pth, Epoch: 50
Leggendo i dati di test...
Test size: 116323
Numero di classi uniche nel dataset di test: 62
Etichette uniche nel dataset di test: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]
Directory per GradCAM: results/results_None_SGD_lr0.01_5_3/KaNet5/GradCAM

Generando Grad-CAM per l'indice: 0
Immagine Originale salvata in: results/results_None_SGD_lr0.01_5_3/KaNet5/GradCAM/sample_0_original.png
Heatmap Grad-CAM salvata in: results/results_None_SGD_lr0.01_5_3/KaNet5/GradCAM/sample_0_heatmap.png
Immagine con Heatmap Sovrapposta salvata in: results/results_None_SGD_lr0.01_5_3/KaNet5/GradCAM/sample_0_overlay.png
Grafico Completo salvato in: results/results_None_SGD_lr0.01_5_3/KaNet5/GradCAM/